In [1]:
import torch

In [2]:
torch.rand(3,2)

tensor([[0.8526, 0.5794],
        [0.3037, 0.6183],
        [0.5525, 0.7777]])

In [3]:
from transformers import AutoformerConfig, AutoformerForPrediction

In [4]:
PATH_TO_RAW_DATA = "/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Gaby/MetaData_ForMike/organized_data/all_photometry_data"